## Import Libraries

In [ ]:
import open3d as o3d
import open3d.core as o3c
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys
from pathlib import Path

# Only needed for tutorial, monkey patches visualization
sys.path.append("..")
import open3d_tutorial as o3dtut
# Change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ 

## Nonsense

In [ ]:
# # ======================== Redwood Data ========================
# # 
# path_images = Path('images')
# 
# # Read the test images
# test_color_raw = o3d.io.read_image(str(path_images / 'test_data/RGBD/color/00000.jpg'))
# test_depth_raw = o3d.io.read_image(str(path_images / 'test_data/RGBD/depth/00000.png'))
# 
# # Convert Open3D images to numpy arrays
# test_color_array = np.asarray(test_color_raw)
# test_depth_array = np.asarray(test_depth_raw)
# 
# # Print out the properties
# print("Test Color Image - Data type:", test_color_array.dtype, ", Shape:", test_color_array.shape)
# print("Test Depth Image - Data type:", test_depth_array.dtype, ", Shape:", test_depth_array.shape)
# print("Min value in Test Depth Image:", np.min(test_depth_array))
# print("Max value in Test Depth Image:", np.max(test_depth_array))


In [ ]:
# ***************** MISCELLANEOUS TESTS ***************** 
"""
# Load the color image again and check its properties
color_raw = o3d.io.read_image(str(path_images / 'small_test_color.png'))
color_array = np.asarray(color_raw)
print("Rechecked Color image - Data type:", color_array.dtype)
print("Rechecked Color image - Shape:", color_array.shape)
print("Rechecked Color image - Min value:", np.min(color_array))
print("Rechecked Color image - Max value:", np.max(color_array))

plt.figure(figsize=(6, 6))
plt.imshow(color_array)
plt.title('Original Color Image')
plt.show()

# Check the depth values in the new RGBDImage
depth_array_from_channel = np.asarray(rgbd_image_from_channel.depth)
print("Depth array info with depth channel:")
print("Min value:", np.min(depth_array_from_channel))
print("Max value:", np.max(depth_array_from_channel))
print("Unique values:", np.unique(depth_array_from_channel))

# Visualize the depth part of the RGBDImage
plt.figure(figsize=(8, 6))
plt.imshow(depth_array_from_channel, cmap='gray')
plt.colorbar()
plt.title('Depth from RGBDImage (Depth Channel)')
plt.show()

# Visualize the depth part of the RGBDImage
plt.figure(figsize=(8, 6))
plt.imshow(depth_array_from_rgb_channel, cmap='gray')
plt.colorbar()
plt.title('Depth from RGBDImage (RGB Channel)')
plt.show()

# Visualize the depth part of the RGBDImage
plt.figure(figsize=(8, 6))
plt.imshow(depth_array_from_rgb_channel, cmap='gray')
plt.colorbar()
plt.title('Depth from RGBDImage (RGB Channel)')
plt.show()

# Check the depth values in the new RGBDImage
depth_array_from_rgb_channel = np.asarray(rgbd_image_from_rgb_channel.depth)
print("Depth array info with RGB channel:")
print("Min value:", np.min(depth_array_from_rgb_channel))
print("Max value:", np.max(depth_array_from_rgb_channel))
print("Unique values:", np.unique(depth_array_from_rgb_channel))

# Check the depth values in the new RGBDImage
depth_array_from_rgb_channel = np.asarray(rgbd_image_from_rgb_channel.depth)
print("Depth array info with RGB channel:")
print("Min value:", np.min(depth_array_from_rgb_channel))
print("Max value:", np.max(depth_array_from_rgb_channel))
print("Unique values:", np.unique(depth_array_from_rgb_channel))

# Confirm the conversion
print("RGB Color image - Data type:", color_array_rgb.dtype)
print("RGB Color image - Shape:", color_array_rgb.shape)
print("RGB Color image - Min value:", np.min(color_array_rgb))
print("RGB Color image - Max value:", np.max(color_array_rgb))
"""


## Raw Shaders Depth 

In [ ]:

# Raw screenshot & Shaders Depth
path_images = Path('images')

color_raw = o3d.io.read_image(str(path_images / 'small_test_color.png'))
depth_raw = o3d.io.read_image(str(path_images / 'small_test_depth.png'))

color_array = np.asarray(color_raw)
depth_array = np.asarray(depth_raw)

# Ensure we are using the original depth values
original_depth_array = np.asarray(depth_raw)

# Extract channel 0 as the depth channel
depth_channel = original_depth_array[:, :, 0]

# Ensure depth_channel is C-contiguous and of type 'uint8'
if not depth_channel.flags['C_CONTIGUOUS']:
    depth_channel = np.ascontiguousarray(depth_channel)
if depth_channel.dtype != np.uint8:
    depth_channel = depth_channel.astype(np.uint8)

# Create an Open3D image from the depth channel
depth_channel_raw = o3d.geometry.Image(depth_channel)

# Extract the RGB channels from the RGBA image
color_array_rgb = color_array[:, :, :3]

# Ensure color_array_rgb is C-contiguous
if not color_array_rgb.flags['C_CONTIGUOUS']:
    color_array_rgb = np.ascontiguousarray(color_array_rgb)

# Ensure color_array_rgb is of type 'uint8'
if color_array_rgb.dtype != np.uint8:
    color_array_rgb = color_array_rgb.astype(np.uint8)

# Create an Open3D image from the RGB color array
color_raw_rgb = o3d.geometry.Image(color_array_rgb)

# Create the RGBDImage using the RGB color image and the depth channel
rgbd_image_from_rgb_channel = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw_rgb,
    depth_channel_raw,
    depth_scale=1.0,   # Adjust based on the actual depth scale
    depth_trunc=255,   # Adjust based on the actual depth range you want to consider
    convert_rgb_to_intensity=True
)

plt.subplot(1, 2, 2)
plt.title('depth image')
plt.imshow(rgbd_image_from_rgb_channel.depth)

plt.subplot(1, 2, 1)
plt.title('grayscale image')
plt.imshow(rgbd_image_from_rgb_channel.color)
plt.show()

In [ ]:
# Image dimensions
height, width, _ = color_array_rgb.shape

# FOV and aspect ratio
fov = 70  # in degrees
aspect_ratio = width / height

# Focal length calculation
fx = width / (2 * np.tan(np.deg2rad(fov / 2)))
fy = fx / aspect_ratio

# Principal point (usually the image center)
cx = width / 2
cy = height / 2

# Create PinholeCameraIntrinsic object
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)


# Create point cloud from RGB-D image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image_from_rgb_channel,
    intrinsic
)

# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], zoom=0.5)



## Marigold Depth

In [ ]:
from pathlib import Path
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

path_images = Path('images')

# Read the color and depth images
color_raw = o3d.io.read_image(str(path_images / 'small_test_color.png'))
depth_raw = o3d.io.read_image(str(path_images / 'marigold_depth.png'))

# Convert the Open3D images to numpy arrays
color_array = np.asarray(color_raw)
depth_array = np.asarray(depth_raw)

# Check the range of depth values
print("Depth image - Data type:", depth_array.dtype)
print("Depth image - Shape:", depth_array.shape)
print("Depth image - Min value:", np.min(depth_array))
print("Depth image - Max value:", np.max(depth_array))

# Normalize the depth values for visualization
depth_normalized_for_visualization = (depth_array - np.min(depth_array)) / (np.max(depth_array) - np.min(depth_array))

# Create an Open3D image from the depth array
depth_raw = o3d.geometry.Image(depth_array)

# Extract the RGB channels from the RGBA color image
color_array_rgb = color_array[:, :, :3]

# Ensure color_array_rgb is C-contiguous and of type 'uint8'
if not color_array_rgb.flags['C_CONTIGUOUS']:
    color_array_rgb = np.ascontiguousarray(color_array_rgb)
if color_array_rgb.dtype != np.uint8:
    color_array_rgb = color_array_rgb.astype(np.uint8)

# Create an Open3D image from the RGB color array
color_raw_rgb = o3d.geometry.Image(color_array_rgb)

# Adjust depth_scale and depth_trunc for creating the RGBDImage
depth_scale = 1000.0  # Convert from millimeters to meters if the values are in millimeters
depth_trunc = np.max(depth_array) / depth_scale  # Set depth truncation to the maximum value in the depth image

# Create the RGBDImage using the RGB color image and the depth array
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw_rgb,
    depth_raw,
    depth_scale=depth_scale,
    depth_trunc=depth_trunc,
    convert_rgb_to_intensity=True
)

# Visualize the grayscale (converted from RGB) and depth images from the RGBDImage
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Block-game Grayscale Image')
plt.imshow(np.asarray(rgbd_image.color), cmap='gray')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title('Block-game Depth Image')
plt.imshow(np.asarray(rgbd_image.depth), cmap='gray')
plt.colorbar()

plt.show()


In [ ]:
height, width, _ = color_array_rgb.shape
fov = 70  # in degrees
aspect_ratio = width / height
fx = width / (2 * np.tan(np.deg2rad(fov / 2)))
fy = fx / aspect_ratio
cx = width / 2
cy = height / 2
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

# Create a point cloud from the RGBD image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)

# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], zoom=0.5)

## Anything Depth

In [ ]:
from pathlib import Path
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

path_images = Path('images')

# Read the color and depth images
color_raw = o3d.io.read_image(str(path_images / 'test2_color.png'))
depth_raw = o3d.io.read_image(str(path_images / 'tiktik2_depth.png'))

# Convert the Open3D images to numpy arrays
color_array = np.asarray(color_raw)
depth_array = np.asarray(depth_raw)

# Check the range of depth values
print("Depth image - Data type:", depth_array.dtype)
print("Depth image - Shape:", depth_array.shape)
print("Depth image - Min value:", np.min(depth_array))
print("Depth image - Max value:", np.max(depth_array))

# Invert the depth values
depth_array_inverted = np.max(depth_array) - depth_array

# Normalize the inverted depth values for visualization
depth_inverted_normalized_for_visualization = (depth_array_inverted - np.min(depth_array_inverted)) / (np.max(depth_array_inverted) - np.min(depth_array_inverted))

# Visualize the normalized inverted depth image
plt.figure(figsize=(6, 6))
plt.title('Normalized Inverted Depth Image')
plt.imshow(depth_inverted_normalized_for_visualization, cmap='gray')
plt.colorbar()
plt.show()

# Create an Open3D image from the inverted depth array
depth_inverted_raw = o3d.geometry.Image(depth_array_inverted)

# Extract the RGB channels from the RGBA color image
color_array_rgb = color_array[:, :, :3]

# Ensure color_array_rgb is C-contiguous and of type 'uint8'
if not color_array_rgb.flags['C_CONTIGUOUS']:
    color_array_rgb = np.ascontiguousarray(color_array_rgb)
if color_array_rgb.dtype != np.uint8:
    color_array_rgb = color_array_rgb.astype(np.uint8)

# Create an Open3D image from the RGB color array
color_raw_rgb = o3d.geometry.Image(color_array_rgb)

# Adjust depth_scale and depth_trunc for creating the RGBDImage
depth_scale = 1000.0  # Convert from millimeters to meters if the values are in millimeters
depth_trunc = np.max(depth_array_inverted) / depth_scale  # Set depth truncation to the maximum value in the depth image

# Create the RGBDImage using the RGB color image and the inverted depth array
rgbd_image_inverted = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw_rgb,
    depth_inverted_raw,
    depth_scale=depth_scale,
    depth_trunc=depth_trunc,
    convert_rgb_to_intensity=True
)

# Visualize the grayscale (converted from RGB) and inverted depth images from the RGBDImage
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Grayscale Image')
plt.imshow(np.asarray(rgbd_image_inverted.color), cmap='gray')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title('Inverted Depth Image')
plt.imshow(np.asarray(rgbd_image_inverted.depth), cmap='gray')
plt.colorbar()

plt.show()

# Create point cloud from RGB-D image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image_inverted,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
)

# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], zoom=0.5)
